Imports

In [97]:
import sys
import logging
import json
import numpy as np
import matplotlib.pyplot as plt

Carregar variáveis globais

In [88]:
%store -r id_consulta

Configurando LOGGER

In [89]:
logging.basicConfig(level=logging.INFO, stream=sys.stdout,
                    format='[%(asctime)s]%(levelname)s(%(name)s): %(message)s')
LOGGER = logging.getLogger(__name__)

Carregando .jsons

In [90]:
with open('map_consultas_relevancias.json', 'r', encoding='utf-8') as f:
        map_consultas_relevancias = json.load(f)
with open('resultados_bm25.json', 'r', encoding='utf-8') as f:
    resultados_bm25 = json.load(f)
with open('resultados_tfidf.json', 'r', encoding='utf-8') as f:
    resultados_tfidf = json.load(f)
# CORREÇÃO: Alterado de 'resultados_sorl.json' para 'resultado_solr.json'
with open('resultados_solr.json', 'r', encoding='utf-8') as f:
    resultados_solr = json.load(f)
LOGGER.info("Arquivos carregados com sucesso.")

[2025-07-22 09:43:30,602]INFO(__main__): Arquivos carregados com sucesso.


In [91]:
def obter_documentos_relevantes_verdadeiros(id_consulta, mapa_relevancias_completo):
    # 1. Acessa o dicionário de relevâncias para a consulta específica
    # O uso de .get() evita erros se o ID não existir
    relevancias_da_consulta = mapa_relevancias_completo.get(str(id_consulta))

    # 2. Se a consulta não existir no mapa, retorna uma lista vazia
    if not relevancias_da_consulta:
        return []

    # 3. Cria uma lista apenas com os IDs dos documentos cujo score é 1 ou 2
    documentos_relevantes = [
        doc_id
        for doc_id, score in relevancias_da_consulta.items()
        if score in [1, 2]
    ]
    
    # 4. Retorna a lista de documentos relevantes encontrados
    return documentos_relevantes

Verificar e retornar correspondência entre consultas relevância e resultados

In [92]:
def retornar_correspondencia(resultados_modelo, map_consultas_relevancias):
    # 1. Obtém os documentos relevantes verdadeiros
    documentos_relevantes_verdadeiros_raw = obter_documentos_relevantes_verdadeiros(id_consulta, map_consultas_relevancias)

    # 2. Padroniza os dois conjuntos de dados para string e remove espaços
    # Padroniza os resultados do modelo
    set_resultados_modelo = {str(item).strip() for item in resultados_modelo}
    
    # Padroniza os documentos relevantes
    set_documentos_relevantes = {str(item).strip() for item in documentos_relevantes_verdadeiros_raw}

    # 3. Encontra a intersecção entre os dois conjuntos agora padronizados
    correspondencias = set_resultados_modelo.intersection(set_documentos_relevantes)

    # 4. Retorna a lista de correspondências (acertos)
    return list(correspondencias)

BM25 - Precision, Recall e F1

In [93]:
# Calcula Documentos relevantes verdadeiros
documentos_relevantes_verdadeiros = len(obter_documentos_relevantes_verdadeiros(id_consulta, map_consultas_relevancias))
LOGGER.info(f"Documentos Relevantes Verdadeiros: {documentos_relevantes_verdadeiros} encontrados")

# Calcula acertos
bm25_acertos = len(retornar_correspondencia(resultados_bm25, map_consultas_relevancias))
LOGGER.info(f"Acertos BM25: {bm25_acertos}")

# Calcula Precision
precision_bm25 = bm25_acertos / len(resultados_bm25) * 100 if len(resultados_bm25) > 0 else 0
LOGGER.info(f"Precisão BM25: {precision_bm25} %")

# Calcula Recall
recall_bm25 = bm25_acertos / documentos_relevantes_verdadeiros * 100 if documentos_relevantes_verdadeiros > 0 else 0
LOGGER.info(f"Recall BM25: {recall_bm25} %")

# Calcula F1 Score
f1_bm25 = 2 * (precision_bm25 * recall_bm25) / (precision_bm25 + recall_bm25) if (precision_bm25 + recall_bm25) > 0 else 0
LOGGER.info(f"F1 Score BM25: {f1_bm25} %")  

[2025-07-22 09:43:30,650]INFO(__main__): Documentos Relevantes Verdadeiros: 103 encontrados
[2025-07-22 09:43:30,652]INFO(__main__): Acertos BM25: 13
[2025-07-22 09:43:30,653]INFO(__main__): Precisão BM25: 26.0 %
[2025-07-22 09:43:30,654]INFO(__main__): Recall BM25: 12.62135922330097 %
[2025-07-22 09:43:30,655]INFO(__main__): F1 Score BM25: 16.99346405228758 %


TD-IDF - Precision, Recall e F1

In [94]:
# Calcula Documentos relevantes verdadeiros
documentos_relevantes_verdadeiros = len(obter_documentos_relevantes_verdadeiros(id_consulta, map_consultas_relevancias))
LOGGER.info(f"Documentos Relevantes Verdadeiros: {documentos_relevantes_verdadeiros} encontrados")

# Calcula acertos
tfidf_acertos = len(retornar_correspondencia(resultados_tfidf, map_consultas_relevancias))
LOGGER.info(f"Acertos TFIDF: {tfidf_acertos}")

# Calcula Precision
precision_tfidf = tfidf_acertos / len(resultados_tfidf) * 100 if len(resultados_tfidf) > 0 else 0
LOGGER.info(f"Precisão TFIDF: {precision_tfidf} %")

# Calcula Recall
recall_tfidf = tfidf_acertos / documentos_relevantes_verdadeiros * 100 if documentos_relevantes_verdadeiros > 0 else 0
LOGGER.info(f"Recall TFIDF: {recall_tfidf} %")

# Calcula F1 Score
f1_tfidf = 2 * (precision_tfidf * recall_tfidf) / (precision_tfidf + recall_tfidf) if (precision_tfidf + recall_tfidf) > 0 else 0
LOGGER.info(f"F1 Score TFIDF: {f1_tfidf} %")

[2025-07-22 09:43:30,702]INFO(__main__): Documentos Relevantes Verdadeiros: 103 encontrados
[2025-07-22 09:43:30,703]INFO(__main__): Acertos TFIDF: 14
[2025-07-22 09:43:30,704]INFO(__main__): Precisão TFIDF: 28.000000000000004 %
[2025-07-22 09:43:30,705]INFO(__main__): Recall TFIDF: 13.592233009708737 %
[2025-07-22 09:43:30,707]INFO(__main__): F1 Score TFIDF: 18.30065359477124 %


Sorl - Precision, Recall e F1

In [95]:
# Calcula Documentos relevantes verdadeiros
documentos_relevantes_verdadeiros = len(obter_documentos_relevantes_verdadeiros(id_consulta, map_consultas_relevancias))
LOGGER.info(f"Documentos Relevantes Verdadeiros SOLR: {documentos_relevantes_verdadeiros} encontrados")

# Calcula acertos
sorl_acertos = len(retornar_correspondencia(resultados_solr, map_consultas_relevancias))
LOGGER.info(f"Acertos SOLR: {sorl_acertos}")

# Calcula Precision
precision_sorl = sorl_acertos / len(resultados_solr) * 100 if len(resultados_solr) > 0 else 0
LOGGER.info(f"Precisão SOLR: {precision_sorl} %")

# Calcula Recall
recall_sorl = sorl_acertos / documentos_relevantes_verdadeiros * 100 if documentos_relevantes_verdadeiros > 0 else 0
LOGGER.info(f"Recall SOLR: {recall_sorl} %")

# Calcula F1 Score
f1_sorl = 2 * (precision_sorl * recall_sorl) / (precision_sorl + recall_sorl) if (precision_sorl + recall_sorl) > 0 else 0
LOGGER.info(f"F1 Score SOLR: {f1_sorl} %")

[2025-07-22 09:43:30,722]INFO(__main__): Documentos Relevantes Verdadeiros SOLR: 103 encontrados
[2025-07-22 09:43:30,723]INFO(__main__): Acertos SOLR: 12
[2025-07-22 09:43:30,724]INFO(__main__): Precisão SOLR: 24.0 %
[2025-07-22 09:43:30,726]INFO(__main__): Recall SOLR: 11.650485436893204 %
[2025-07-22 09:43:30,727]INFO(__main__): F1 Score SOLR: 15.686274509803921 %


Função para Calcular Precisão Interpolada

In [ ]:
RECALL_LEVELS = np.linspace(0.0, 1.0, 11)

def calcular_precisao_interpolada(retrieved_ids, relevant_ids_set):
    if not relevant_ids_set:
        return [0.0] * 11

    # Garante que a comparação seja feita entre os mesmos tipos (string)
    retrieved_ids_str = {str(doc_id) for doc_id in retrieved_ids}
    relevant_ids_set_str = {str(doc_id) for doc_id in relevant_ids_set}

    precision_values = []
    recall_values = []
    relevant_found_count = 0
    for i, doc_id in enumerate(retrieved_ids):
        if str(doc_id) in relevant_ids_set_str:
            relevant_found_count += 1
            precision = relevant_found_count / (i + 1)
            recall = relevant_found_count / len(relevant_ids_set_str)
            precision_values.append(precision)
            recall_values.append(recall)

    interpolated_precision = []
    for r_level in RECALL_LEVELS:
        precisions_at_level = [p for r, p in zip(recall_values, precision_values) if r >= r_level]
        interpolated_precision.append(max(precisions_at_level) if precisions_at_level else 0.0)
        
    return interpolated_precision

Cálculo das Curvas e Plotagem do Gráfico

In [ ]:
# Obter a lista de documentos relevantes (como strings)
documentos_relevantes_set = set(obter_documentos_relevantes_verdadeiros(id_consulta, map_consultas_relevancias))

# Dicionário para armazenar as curvas
curvas_modelos = {
    "BM25": calcular_precisao_interpolada(resultados_bm25, documentos_relevantes_set),
    "Vetorial (TF-IDF)": calcular_precisao_interpolada(resultados_tfidf, documentos_relevantes_set),
    "Solr": calcular_precisao_interpolada(resultados_solr, documentos_relevantes_set)
}

# Plotar o gráfico
LOGGER.info("Plotando o gráfico de Precisão x Recall...")
plt.figure(figsize=(12, 8))

for nome_modelo, curva in curvas_modelos.items():
    plt.plot(RECALL_LEVELS, curva, marker='o', linestyle='-', label=nome_modelo)

plt.title(f'Precisão x Recall Interpolado (Consulta {id_consulta})', fontsize=16)
plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precisão', fontsize=12)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend(fontsize=12)
plt.xticks(RECALL_LEVELS)
plt.yticks(np.linspace(0.0, 1.0, 11))
plt.ylim(0, 1.05)
plt.xlim(-0.01, 1.01)
plt.show()